In [17]:
import os
import torchio as tio
import numpy as np
from tifffile import imread
from tifffile import imsave

In [18]:
def transform_img_msk(
    idx,
    imgs,
    img_dir,
    msk_dir,
    transform,
):
    img_fname = imgs[idx]
    img_path = os.path.join(img_dir, img_fname)
    msk_path = os.path.join(msk_dir, img_fname)
    print(img_path)
    # load mask and image
    img = imread(img_path)
    msk = imread(msk_path)
    assert img.shape == msk.shape, '[error] expected img and msk sizes must match, img {}, msk {}'.format(img.shape, msk.shape)

    # change data type from np.uint16 to np.int16
    img = img.astype(np.float32)
    msk = msk.astype(np.float32)

    # crop if training
    # if self.train: # TODO: remove resize during inference
    # img, msk = self.resize(img, msk)

    # add channel
    img = np.expand_dims(img, 0)
    msk = np.expand_dims(msk, 0)

    # transform with TorchIO
    tmp_sub = transform(tio.Subject(img=tio.ScalarImage(tensor=img), msk=tio.LabelMap(tensor=msk)))
    img, msk = tmp_sub.img.numpy()[0].astype(np.uint16), tmp_sub.msk.numpy()[0].astype(np.uint16)

    return img, msk




In [19]:
# define transform
spacing = (1.0, 1.0, 0.5159651630825821)
spacing = np.flip(spacing) # CAREFUL: spacing must be in ZYX and not in XYZ!

crop_shape = (128,128,128)

transform = tio.Compose([
            tio.Resample(spacing),
            tio.CropOrPad(crop_shape)
])

# define list of images
#img_dir = '../data/nucleus/images_sophie'
#msk_dir = '../data/nucleus/masks_sophie'
img_dir = '/home/mezquitap/chomocentres/images'
msk_dir = '/home/mezquitap/chomocentres/chromocenter_masks'
imgs = os.listdir(img_dir)


for idx in range(len(imgs)):
    img_resampled, msk_resampled = transform_img_msk(
        idx=idx,
        imgs=imgs,
        img_dir=img_dir,
        msk_dir=msk_dir,
        transform=transform,
    )
    # save the images
    imsave('/home/mezquitap/chomocentres/reshape/img/'+imgs[idx], img_resampled)
    imsave('/home/mezquitap/chomocentres/reshape/mask/'+imgs[idx], msk_resampled)  
    
    # TODO:
    
    

/home/mezquitap/chomocentres/images/KAKU4-wt--CRWN1-wt--CRWN4-wt_Cot_J13_STD_FIXE_H258_W1_0_C0.tif
/home/mezquitap/chomocentres/images/KAKU4-wt--CRWN1-wt--CRWN4-wt_Cot_J13_STD_FIXE_H258_T1_28_C0.tif
/home/mezquitap/chomocentres/images/KAKU4-wt--CRWN1-wt--CRWN4-wt_Cot_J13_STD_FIXE_H258_T1_9_C0.tif
/home/mezquitap/chomocentres/images/KAKU4-wt--CRWN1-wt--CRWN4-wt_Cot_J13_STD_FIXE_H258_R1_12_C0.tif
/home/mezquitap/chomocentres/images/KAKU4-wt--CRWN1-wt--CRWN4-wt_Cot_J13_STD_FIXE_H258_W1_25_C0.tif
/home/mezquitap/chomocentres/images/20220122_1581412312099_Ath_Col0--HON4-wt_Cot_D13_STD_FIXE_H258_I1_w11 DAPI SIM_00.tif
/home/mezquitap/chomocentres/images/KAKU4-wt--CRWN1-wt--CRWN4-wt_Cot_J13_STD_FIXE_H258_X1_6_C0.tif
/home/mezquitap/chomocentres/images/KAKU4-wt--CRWN1-wt--CRWN4-wt_Cot_J13_STD_FIXE_H258_W1_2_C0.tif
/home/mezquitap/chomocentres/images/20200204_1581418010044_Ath_Col0--KAKU4-wt_Cot_D13_STD_FIXE_H258_A3_w11 DAPI SIM variable_2_C0.tif
/home/mezquitap/chomocentres/images/KAKU4-wt--CR